In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from transformers import RobertaTokenizer
import torch
from tqdm import tqdm
torch.cuda.empty_cache()
import numpy as np
from sklearn.metrics import f1_score

In [2]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [11]:
df.loc[df['polarity']=='POSITIVE']

,Unnamed: 0,text,subjectivity,polarity
0,0,Love our President,OPINIONATED,POSITIVE
2,2,Thank God we Parlerðâï¸ðºð¸.I'm grat...,OPINIONATED,POSITIVE
12,12,"I admire you tremendously! Smart, quick witted...",OPINIONATED,POSITIVE
15,15,Love it!,OPINIONATED,POSITIVE
20,20,THIS IS SO AMAZING I THINK I AM GOING TO BREAK...,OPINIONATED,POSITIVE
...,...,...,...,...
1182,1182,Simple yet brilliant,OPINIONATED,POSITIVE
1188,1188,PROUD TO BE AN AMERIKKKAN!,OPINIONATED,POSITIVE
1189,1189,"So true, especially today.",OPINIONATED,POSITIVE
1190,1190,THIS IS BEAUTIFUL,OPINIONATED,POSITIVE


In [18]:
labels_dict = {'NEUTRAL':0, 'POSITIVE':1, 'NEGATIVE':2}
df = pd.read_csv('../data/cleaned_subj_polar__.csv', encoding='ISO-8859-1')
positive_df = df.loc[df['polarity']=='POSITIVE']
negative_df = df.loc[df['polarity']=='NEGATIVE']
neutral_df = df.loc[df['subjectivity']=='NEUTRAL'].sample(n=300, replace=False)
neutral_df['polarity'] = 'NEUTRAL'

df = pd.concat([positive_df, neutral_df, negative_df]).sample(frac=1).reset_index(drop=True)[['text', 'polarity']]
df

,text,polarity
0,Meanwhile in Oregon...,NEUTRAL
1,Psalm 2:1-12 God's Son Shall Reign1 Why do the...,NEUTRAL
2,Love this chyron!,POSITIVE
3,Hmmmm.... itâs the LAW!,NEUTRAL
4,Beautiful!!!,POSITIVE
...,...,...
895,There is no reason to be using a writing pen/ ...,NEGATIVE
896,Our vote will never make a difference again. W...,NEGATIVE
897,let's make this go viral by tomorrow,NEUTRAL
898,SOME GOOD NEWS!,POSITIVE


In [20]:
df['label'] = df['polarity'].apply(lambda x: labels_dict[x])
print(len(df))
df.head(10)

900


,text,polarity,label
0,Meanwhile in Oregon...,NEUTRAL,0
1,Psalm 2:1-12 God's Son Shall Reign1 Why do the...,NEUTRAL,0
2,Love this chyron!,POSITIVE,1
3,Hmmmm.... itâs the LAW!,NEUTRAL,0
4,Beautiful!!!,POSITIVE,1
5,Great!!!,POSITIVE,1
6,Make her famous,POSITIVE,1
7,What a pathetic mess,NEGATIVE,2
8,Love George Carlin!,POSITIVE,1
9,THE BIG REVEAL THIS WAS HOW THEY DID IT!!! MAG...,NEUTRAL,0


In [21]:
text = list(df.text)
labels = list(df.label)
train_texts, val_texts, train_labels, val_labels = train_test_split(text, labels, test_size=.15, stratify=labels)

In [22]:
from torch.utils.data import TensorDataset
from transformers import BertTokenizer

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [24]:
train_encode = tokenizer.batch_encode_plus(
    train_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)

valid_encode = tokenizer.batch_encode_plus(
    val_texts,
    add_special_tokens=True,
    padding=True,
    truncation = True,
    max_length = 256,   
    return_tensors='pt'
)
valid_input = valid_encode['input_ids']
valid_attention = valid_encode['attention_mask']
valid_labels = torch.tensor(val_labels)

input_ids = train_encode['input_ids']
attention_mask = train_encode['attention_mask']
labels = torch.tensor(train_labels)


In [25]:
train_data = TensorDataset(input_ids,
                          attention_mask,
                          labels)

valid_data = TensorDataset(valid_input,
                          valid_attention,
                          valid_labels)

In [26]:
from transformers import BertForSequenceClassification

#Finetuning pretrained BERT model with objective -> Sentence Classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data,
                              sampler = RandomSampler(train_data),
                              batch_size = BATCH_SIZE)
valid_dataloader = DataLoader(valid_data,
                              sampler = SequentialSampler(valid_data),
                              batch_size = BATCH_SIZE)


In [28]:
from transformers import AdamW, get_cosine_schedule_with_warmup

#Works better than torch.optim.Adam
optimizer = AdamW(model.parameters(),
                           lr=1e-5,
                           eps=1e-6,
                           weight_decay=1e-1)
#chosen after setting = 3 (f1 -> 0.71), 5 (f1 -> 0.77) but acpc for many classes is 0
#10 Epochs helps with generalization
EPOCHS = 10 

#Works better than torch.optim.CosineAnnealingLR and transformers.get_linear_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [29]:
def f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [30]:
#Per Class Accuracy
def acpc(preds, labels):
    code_dict = {val:key for key,val in labels_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for idx in np.unique(labels_flat):
        y_hat = preds_flat[labels_flat==idx]
        y = labels_flat[labels_flat==idx]
        print(f'Class: {code_dict[idx]}')
        print(f'Accuracy: {len(y_hat[y_hat==idx])/ len(y)}\n')

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [32]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in valid_dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [34]:
vloss = []
score = []
for epoch in tqdm(range(1, EPOCHS+1)):
    model.train()

    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids' : batch[0],
                  'attention_mask': batch[1],
                  'labels' : batch[2]
                  }
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    torch.save(model.state_dict(), f'./bert subj_pol/BERT_subj_pol_epoch_{epoch}.pt')

    tqdm.write(f'\nEpoch {epoch}')

    avg_training_loss = total_loss/len(train_dataloader)            
    tqdm.write(f'Training loss: {avg_training_loss}')

    val_loss, predictions, actual = evaluate(valid_dataloader)
    score_f1 = f1(predictions, actual)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {score_f1}')
    vloss.append(val_loss)
    score.append(score_f1)

Epoch 1:  33%|█████████████████▎                                  | 32/96 [00:17<00:34,  1.87it/s, training_loss=0.294]


Epoch 1:  68%|███████████████████████████████████▏                | 65/96 [00:35<00:17,  1.75it/s, training_loss=0.206]


  0%|                                                                                           | 0/10 [00:53<?, ?it/s]


Epoch 1
Training loss: 0.7193664830798904


Epoch 2:   0%|                                                                                  | 0/96 [00:00<?, ?it/s]

...

Epoch 10:  69%|███████████████████████████████████                | 66/96 [00:38<00:16,  1.78it/s, training_loss=0.003]


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [09:22<00:56, 56.09s/it]


Epoch 10
Training loss: 0.007923349791478055


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:23<00:00, 56.40s/it]

Validation loss: 0.19217882056117935
F1 Score (Weighted): 0.9551600354301031


In [35]:
for i, pair in enumerate(zip(vloss, score)):
    print(i, pair[0], pair[1])

0 0.4175184338408358 0.7688762850053174
1 0.19944526430438547 0.933066933066933
2 0.16656179769950755 0.9329995015388275
3 0.20145461971268935 0.9473912132044663
4 0.1850465912207523 0.9551600354301031
5 0.1879765129483798 0.9551600354301031
6 0.1915614493739079 0.9551600354301031
7 0.19338054648217032 0.9551600354301031
8 0.19203355533125646 0.9551600354301031
9 0.19217882056117935 0.9551600354301031


In [36]:
model.to(device)
model.load_state_dict(torch.load('./bert subj_pol/BERT_subj_pol_epoch_5.pt', map_location=torch.device('cpu')))

_, predictions, actual = evaluate(valid_dataloader)
print(acpc(predictions, actual))
labs = {v:k for k,v in labels_dict.items()}
for i in range (len(predictions)):
    print(labs[np.argmax(predictions[i])], val_texts[i])
    print()

Class: NEUTRAL
Accuracy: 0.8888888888888888

Class: POSITIVE
Accuracy: 1.0

Class: NEGATIVE
Accuracy: 0.9777777777777777

None
NEUTRAL Here is your culprit, catch him and I bet he sings like a canary..

POSITIVE This is incredible!! #1776worldwide

NEGATIVE The media is grotesque. All of a sudden they reacting with false horror. Now they have the courage to use words like âviolenceâ âanarchyâ and âinsurrectionâ have they even checked to see if all of this is a staged event by #Antifa?

POSITIVE Awesome Thank you

POSITIVE This makes me so happy

NEUTRAL Isnt that the truth!! Keep posting please of what is going on!! And stay safe!!

POSITIVE He is so right.

NEGATIVE Communist have taken over DC! How pathetic what our nation's capital has turned into thanks to the DemoRats and RINOs.

NEGATIVE I beg Vernon Jones to please reconsider joining the Republican party. That will be a big waste of your talent. You really need to check on and investigate maybe starting your own poli